In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory of 'src' to the Python path
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Longman phrase and anki deck generation
A more memorable way to learn the core vocabulary as defined in Longman Communications vocab lists, we take the vocab and use an LLM
to generate phrases using it.

## Longman 1000, 2000 and 3000 already provided
Enlish phrases for the longman vocab have already been created and can be found in the 'data' folder

In [ ]:
from src.utils import get_longman_verb_vocab_dict
from src.phrase import generate_phrases_from_vocab_dict

file_path = '../data/longman-communication-3000.json'
vocab_dict = get_longman_verb_vocab_dict(file_path, "S3") #S1 = 1st 1000 words used in Speech, options are S1-3 and W1-3

In [ ]:
#uses LLM calls - it sometimes generates phrases terminated too early (e.g. Mind the pot on the), so advise you scan through and check

#english phrases only initially
longman_phrases = generate_phrases_from_vocab_dict(vocab_dict, max_iterations=15)

In [ ]:
with open("../outputs/longman/longman_1000_phrases.txt", "w", encoding="utf-8") as f:
    for phrase in longman_phrases:
        f.write(phrase + "\n")

## Create an Anki deck from thoses phrases

Or, load one of the english Longman phrase lists in data/ already created and apply to your language

This function:
1. translates
2. generates audio using text to speech
3. packages up the text and audio into several anki decks (in batches), that can be imported into Anki.

The deck_name will is used to derive the deck_id and so despite there being several *.apkg files created, these will all merge successfully into the same deck


In [ ]:

from src.anki_tools import create_anki_deck_from_english_phrase_list, export_to_anki_with_images
from src.generate import ( 
    add_practice_phrases,
    create_story_plan_and_dialogue,
    add_translations,
    add_audio,
    create_album_files)

from src.utils import add_image_paths, test_image_reading



In [ ]:

_ = await create_anki_deck_from_english_phrase_list(example_phrases, deck_name="Longman 1000 - Basque", anki_filename_prefix="longman_1000_basque", batch_size=50)

## Get anki flash card data


In [ ]:
from src.anki_tools import convert_anki_to_story_dict, print_deck_info
# Example usage
collection_path =r"C:\Users\i5\AppData\Roaming\Anki2\User 1\collection.anki2"  # Your Anki database path
deck_name = "Longman 1000 - Swedish"

# Convert the Anki deck to story_dict format
story_dict = convert_anki_to_story_dict(collection_path, deck_name)

In [ ]:
from src.utils import create_test_story_dict


test_dict = create_test_story_dict(story_dict, 1, 2, 3)

In [ ]:
test_phrases = []
for english, target in test_dict['part_1']['translated_phrase_list']:
    test_phrases.append(english)

In [ ]:
test_phrases

# Generate Images for Longman Phrases

In [20]:
from src.utils import create_image_generation_prompt, generate_image_imagen, add_images_to_phrases, clean_filename, save_json, generate_image_stability, load_json, generate_image_deepai
import time
from src.config_loader import config



In [24]:

with open("../data/longman_1000_phrases.txt", "r") as f:
    LONGMAN_PHRASES = [line.strip() for line in f.readlines()]
PROMPT_DICT = load_json("../data/longman_phrase_images/phrase_image_dict.json")

In [ ]:
new_prompt_dict = add_images_to_phrases(LONGMAN_PHRASES[100:200], output_dir="../data/longman_phrase_images")


 37%|███▋      | 37/100 [00:00<00:00, 276.69it/s]

100%|██████████| 100/100 [00:00<00:00, 294.56it/s]

In [ ]:

PROMPT_DICT.update(new_prompt_dict)
save_json(PROMPT_DICT, "../data/longman_phrase_images/phrase_image_dict.json")

In [ ]:
from openai import OpenAI
client = OpenAI()

openai_image = client.images.generate(
  model="dall-e-3",
  prompt="A stressed woman frantically typing on a laptop, surrounded by stacks of papers, with a large calendar nearby showing \"Next Week\" circled in red. In the style of a children's book illustration, Colored pencil sketch, shaded textures, pastel palette",
  n=1,
  size="1024x1024",
  style="natural",
  response_format="b64_json"
)

import base64
import io
from PIL import Image


image_data = base64.b64decode(openai_image.data[0].b64_json)
        
        # Convert to PIL Image
image = Image.open(io.BytesIO(image_data))
image.show()